# Music streaming service

**Research goal** — to check three hypothesis:
1. User activity depends on day of the week. This dependency is diffirent in Moscow and St.Petersburg.
2. On Monday mornings in Moscow some genres dominate, while in St.Petersburg the others prevail.
3. Citizens of Moscow and St.Petersburg prefer different music genres. In Moscow it is usually pop music, in St.Petersburg - russian rap.

**Reserach plan**

 1. Data review
 2. Data pre-processing
 3. Hypothesis testing



## Data review




In [7]:
# importing pandas library
import pandas as pd

In [8]:
# reading the file
df = pd.read_csv('yandex_music_project.csv')

In [9]:
# displaying first 10 lines from the file
display(df.head(10))

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [10]:
# receiving overall information about the data in the table
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


There are 7 columns in the table. All columns contain `object` type of data.

There are several mistakes in style of column namings:
1. Uppercase mixed with lowercase
2. Empty spaces
3. CamelCase is used instead of snake_case


Number of entries differs from column to column which means that there are some missing values.


**Conclusion**

Each line of the table contains information on listened track. Some columns describe the tack (name, artist, genre) and others contain information on the user (city, time of listening).

Preliminary we can state that there is enough data for hypothesis check. But there are some missings and the column namings should be corrected.

## Data pre-processing

### Headings style

In [11]:
# table df headings
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Columns should be renamed in such way:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [12]:
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

In [13]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Missing values

In [14]:
# counting missing values
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Not all of the missing values affect our research. For example, missings in `track` and `artist` do not affect. We can substitute them with some designation.

But missings in `genre` would however interfere with comparisson of music preferences in Moscow and St.Petersburg. Since this is a study project we can't determine the reason behind these missing values. 
* substitute them with some designation,
* estimate how they will affect calculations.

In [15]:
# substitution of missing values to 'unknown'
columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [16]:
# check
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Duplicates

In [17]:
# determining number of duplicates
df.duplicated().sum()

3826

In [18]:
# deleting duplicates
df = df.drop_duplicates().reset_index(drop=True)

In [19]:
# check
df.duplicated().sum()

0

Now we will get rid of implicit duplicates in the `genre` column, for example: the name of the same genre can be written differently.

In [20]:
# Displaying unique genre names
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [21]:
# Function for substitution wrong names with correct ones
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

In [22]:
# Example with 'hiphop' genre:
# List of wrong names
duplicates = ['hip', 'hop', 'hip-hop']

#Correct naming
correct_name = 'hiphop'

replace_wrong_genres(duplicates, correct_name)

In [23]:
# Check
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Conclusion**

Table headers were corrected. Missing values were substituted with `'unknown'`. 

## Hypothesis testing

### Comparisson of user activity in two cities 

To confirm that listening activity between citizens of two cities is different we will check the assumption based on the date from three days: monday, wednesday and friday:

* Divide all of the users into two groups based on the city (Moscow and St.Petersburg)
* Compare each group: how many tracks each group listened to on monday, wednesday and friday.


In [24]:
# Total amount of tracks per city
df_city = df.groupby('city')['track'].count()
display(df_city)

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

Moscow is leading with quantity of tracks listened to. But that doesn't mean that that in Moscow users use the service more often, it means that there are more users there.

In [25]:
# Amount of tracks per day
df_days = df.groupby('day')['track'].count()
display(df_days)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

On average users are less active on Wednesdays. But the picture may change if we closely look on each group separately.

In [26]:
# Function that filters out lines with necessary day and city and returns the number of users

def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [27]:
# number of listenings in Moscow on Monday
display(number_tracks('Monday', 'Moscow'))

15740

In [28]:
# number of listenings in St.Petersburg on Monday
display(number_tracks('Monday', 'Saint-Petersburg'))

5614

In [29]:
# number of listenings in Moscow on Wednesday
display(number_tracks('Wednesday', 'Moscow'))

11056

In [30]:
# number of listenings in St.Petersburg on Wednesday
display(number_tracks('Wednesday', 'Saint-Petersburg'))

7003

In [31]:
# number of listenings in Moscow on Friday
display(number_tracks('Friday', 'Moscow'))

15945

In [32]:
# number of listenings in St.Petersburg on Friday
display(number_tracks('Friday', 'Saint-Petersburg'))

5895

In [33]:
# Results in a table view
track_columns = ['city', 'monday', 'wednesday', 'friday']
track_data = [
    ['Moscow',15740, 11056, 15945],
    ['Saint-Petersburg', 5614, 7003, 5895]
]
display(pd.DataFrame(data=track_data, columns=track_columns))

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Conclusion**

Data represents differences in user behavior:

- In Moscow maximum quantity of listenings is on Monday and Friday.
- In St.Petersburg the situation is vice versa: maximum activity is on Wednesday.

The results support the first hypothesis.

### Music at the beginning and at the end of the week

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [34]:
# saving all the users from Moscow into variable moscow_general
moscow_general = df[df['city'] == 'Moscow'].reset_index(drop=True)
display(moscow_general.head(10))


,user_id,track,artist,genre,city,time,day
0,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
1,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
2,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
3,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
4,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
5,BC5A3A29,Gool la Mita,Shireen Abdul Wahab,world,Moscow,14:08:42,Monday
6,8B5192C0,Is There Anybody Out There? (Panoramic Paralys...,Pink Floyd Floydhead,electronic,Moscow,13:47:49,Monday
7,FF3FD2BD,Мина син генэ кирэк,Ильдар Хакимов,pop,Moscow,09:19:49,Monday
8,CC782B0F,After School Special,Detroit Grand Pubahs,dance,Moscow,20:04:12,Friday
9,E3C5756F,unknown,unknown,unknown,Moscow,09:24:51,Monday


In [35]:
# saving all the users from St.Petersburg into variable spb_general
spb_general = df[df['city'] == 'Saint-Petersburg'].reset_index(drop=True)
display(spb_general.head(10))

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
2,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
3,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
4,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
5,94EB25C2,Make Love Whenever You Can,Arabesque,pop,Saint-Petersburg,13:22:08,Wednesday
6,81D05C7D,SLAVES OF FEAR,HEALTH,alternative,Saint-Petersburg,20:54:48,Monday
7,39DE290E,Веснянка,Веданъ Колодъ,folk,Saint-Petersburg,14:36:47,Monday
8,58AE138A,Pat-a-Cake,Mother Goose Club,children,Saint-Petersburg,20:04:56,Monday
9,B8E29C9C,Самота,unknown,folk,Saint-Petersburg,20:20:49,Wednesday


Создайте функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [36]:
# Function returns the most popular genres in particular day and time
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1] & (genre_df['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['track'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)

SyntaxError: unmatched ']' (3950848454.py, line 4)

In [ ]:
# top-10 genres for Moscow between 7am and 11am
display(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))

In [ ]:
# top-10 genres for St.Petersburg between 7am and 11am
display(genre_weekday(spb_general, 'Monday', '07:00', '11:00'))

In [ ]:
# top-10 genres for Moscow between 5pm and 11pm
display(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))

In [ ]:
# top-10 genres for St.Petersburg between 5pm and 11pm
display(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

**Conclusion**

For Monday morning we have following results:

1. In both cities people listen to similar music. The only difference is that genre "world" made it to top-10 rating in Moscow and "jazz" and "classical" to top-10 rating in St.Petersburg.

2. For Moscow there are lots of missing values - `'unknown'` is on 10th place of the rating. It means that missing values make it a significant portion of data and putting calculations at risk.

For Friday evening the situation is practically the same.

So the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning and at the end of the week.
* Difference between two cities is barely vivid. 

### Genre preferences in two cities


Сгруппируйте таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортируйте результат в порядке убывания и сохраните его в таблице `moscow_genres`.

In [ ]:
# grouping Moscow users by genre
moscow_genres = moscow_general.groupby('genre')['track'].count().sort_values(ascending = False)

In [ ]:
# displaying first 10 lines of moscow_genres
display(moscow_genres.head(10))

In [ ]:
# grouping St.Petersburg users by genre
spb_genres = spb_general.groupby('genre')['track'].count().sort_values(ascending = False)

In [ ]:
display(spb_genres.head(10))

**Conclusion**

Hypothesis is partially confirmed:
* Pop music is the most popular genre in Moscow
* Rap music is equally popular in both cities.


## Results

1. There is a correlation between weekdays and users activity in both cities

2. Genre preferences barely change throughout the week in both cities.
* genre “world” made it to the top-10 in Moscow,
* "gazz" and "classical" are in top-10 for St.Petersburg.

3. Moreover, genre preferences in both cities are very similar